# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$. In the 2D Environment, a car has a probability of $z$ of turning at an intersection when its clock goes off.

### Choose global variables:

In [208]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, p_auto, auto_buffer, theta, gamma, gamma_sd, p, z, seed):
    return [n, alpha, beta, p_auto, auto_buffer, theta, gamma, gamma_sd, p, z, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), p_auto=(0.0, 1), auto_buffer=(1, 5), theta=(0.0, 0.5), gamma=(0.0, 2.0), gamma_sd=(0.0, 0.5), p=(0.0, 1.0), z=(0.0, 1.0), seed=(1, 999))
display(params)

A Jupyter Widget

### Set global variables:
(Note: will break if you set alpha = 0 due to 0 division error)
Remember to rerun this cell every time a slider is updated

In [209]:
n, alpha, beta, p_auto, auto_buffer, theta, gamma, gamma_sd, p, z, seed = params.result

if auto_buffer > n - 1:
    print("[Warning] Buffer for autonomous cars cannot be greater than road length - 1. Defaulting to 1 instead")
    auto_buffer = 1

### Define particle and environment classes:

In [210]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

def format_func(x):
    if x == ' ':
        return x
    if type(x) == str and "A" in x:
        return x
    elif int(x) == 0:
        return "_"
    elif int(x) == 1:
        return "\x1b[34mR\x1b[0m"
#     else:
#         return "R"
#     elif type(x) == str:
#         return x
#     else:
#         return "R"
    
np.set_printoptions(formatter={'object': format_func})


class Particle:
    
    count = 0
    symbol = '1'
    
    def __init__(self, time, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        self._time = time # time at which the particle went to this position
        
    def update_position(self, time, value=1):
        self._position += value
        self._time = time # update time
        
    def get_position(self):
        return self._position
    
class AutoParticle(Particle):
    
    symbol = '\x1b[31mA\x1b[0m' #trying color
    
    def __init__(self, time, buffer_ahead=1):
        super().__init__(time)
        self.buffer_ahead = buffer_ahead
        
    def should_move(self, road):
        ahead = road[self._position + 1 : min(len(road), self._position + 1 + self.buffer_ahead + 1)]
        behind = road[self._position - 1] if self._position > 0 else 0
        
        if not ahead or ahead[0] != 0:
            # At end of road, or there is a car immediately in front. Cannot move.
            return False
        
        # Whether there are any cars within `buffer_ahead` spaces in front of this one,
        # assuming the space immediately in front is open.
        car_ahead = any(ahead[1:])
        
        if not car_ahead or behind != 0:
            # Only consider it safe to move if the next `buffer_ahead` spaces are all empty.
            # The exception is if there is a car immediately behind this one — if so, then in order
            #  to avoid causing slowdowns, it's okay to move forward.
            return True
        
        
class Environment:
    #np.set_printoptions(formatter={'str_kind': lambda x: x}) # shouldn't need this
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.auto_particles = []
        self.occupied_positions = [0 for _ in range(n)] #trying to keep empty unless road
        
    def move_auto_particles(self, prev_time, new_time, cell_speeds, sg_stopped, move_opps, curr_time, actions):
        # Auto particles move according to expected hitting time of ordinary cars, without any noise
        interval = int(round(1 / gamma))
        
        # Start at nearest multiple of `interval` that is > previous time
        time = (prev_time + interval) // interval * interval
        
        while time <= new_time:
            for particle in self.auto_particles:
                if particle.should_move(self.occupied_positions):
                    self.move_particle(particle, cell_speeds, sg_stopped, move_opps, time, actions)
                elif particle.should_move(self.occupied_positions) == False: # could be None
                    sg_stopped[0] += 1
                move_opps[0] += 1
            time += interval
    
    def move_particle(self, particle, cell_speeds, sg_stopped, move_opps, curr_time, actions):
        particle_pos = particle.get_position()
        self.occupied_positions[particle_pos] = 0
        self.occupied_positions[particle_pos + 1] = particle.symbol
        cell_speeds.append(1 / (curr_time - particle._time))
        particle.update_position(curr_time)
        actions += [(curr_time, particle.id, particle.get_position())]
    
    def run_simulation(self, time_len=60):
        Particle.count = 0
        curr_time = 0
        actions = []
        entrances = 0
        exits = 0
        travel_times = []
        cell_speeds = []
        move_opps = [0]
        sg_stopped = [0]
        num_cars = [0]
        time_of_cars = []
        stop_and_go_time = 0
        sg_time_lapse = 0 
        sg_interval_len = 5 # constant
        sg_max_stoppage_percent = .3 #constant
        # sg_speeds = []
        #sg_threshold = alpha * gamma # constant
        
        while curr_time < time_len:
            clear_output(wait=True)
            #print(self.occupied_positions)
            #print(np.array2string(np.asarray(self.occupied_positions, dtype='object'), separator=',', formatter={'object': format_func})) #1d print fix??
            print(np.array2string(np.asarray(self.occupied_positions, dtype='object'), formatter={'object': format_func})) 
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            self.move_auto_particles(curr_time, curr_time + time_lapse, cell_speeds, sg_stopped, move_opps, curr_time, actions)
            
            #stop and go IN 1D
            if sg_time_lapse != 0 and sg_time_lapse >= sg_interval_len:
                if move_opps[0] > 0 and sg_stopped[0] / move_opps[0] >= sg_max_stoppage_percent:
                        stop_and_go_time += sg_time_lapse
                sg_time_lapse = 0
                sg_stopped = [0]
                move_opps = [0]
            else:
                sg_time_lapse += time_lapse
            
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    if np.random.random() < p_auto:
                        new_particle = AutoParticle(curr_time, buffer_ahead=auto_buffer)
                        self.auto_particles.append(new_particle)
                    else:
                        new_particle = Particle(curr_time, gamma=np.random.normal(gamma, gamma_sd), p=p)
                        self.particles.append(new_particle)
                
                    self.occupied_positions[0] = new_particle.symbol
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    travel_times.append([curr_time, None])
                    num_cars.append(len(self.particles) + len(self.auto_particles))
                    time_of_cars.append(curr_time)
                     
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]:
                    if self.occupied_positions[self.n-1] == Particle.symbol:
                        end_particle = self.particles.pop(0)
                    elif self.occupied_positions[self.n-1] == AutoParticle.symbol:
                        end_particle = self.auto_particles.pop(0)
                    else:
                        raise Exception("Unknown symbol found at end of road:", self.occupied_positions[self.n-1])
                    
                    self.occupied_positions[self.n-1] = 0
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    travel_times[end_particle.id][1] = curr_time
                    num_cars.append(len(self.particles) + len(self.auto_particles))
                    time_of_cars.append(curr_time)
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                move_opps[0] += 1
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] != 0: # changed to refer to self.n instead of n
                    if not particle_pos == self.n - 1 and self.occupied_positions[particle_pos + 1] != 0:
                        sg_stopped[0] += 1
                    continue
                else:
                    self.move_particle(particle, cell_speeds, sg_stopped, move_opps, curr_time, actions)
#                     self.occupied_positions[particle_pos] = 0
#                     self.occupied_positions[particle_pos + 1] = particle.symbol
                    
#                     cell_speeds.append(1 / (curr_time - particle._time))
#                     particle.update_position(curr_time)
#                     actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        #print(np.array2string(self.occupied_positions, separator=',', formatter={'str_kind': lambda x: x})) # took away quotes
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
        time_of_cars.append(curr_time)
        t = len(time_of_cars)
        for i in range(1, t):
            time_of_cars[t - i] = time_of_cars[t - i] - time_of_cars[t - i - 1]
        average_num_cars = np.average(num_cars, weights=np.array(time_of_cars) / curr_time)
        print('Average number of cars: {}'.format(round(average_num_cars, 2)))
        print('Stop and Go Time Percentage: {}'.format(stop_and_go_time/time_len))
        
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [220]:
# direction is 0 = horizontal, 1 = vertical

class Particle2D:
    
    num_particles = 0
    
    def __init__(self, road, direction):
        
        self.id = Particle2D.num_particles
        Particle2D.num_particles += 1
        
        self.gamma = gamma
        self.p = p
        self.z = z
        
        self.position = [0, road.position] if not road.direction else [road.position, 0]
        self.direction = road.direction
        self.active = True
        self.time = 0
    
    def update_position(self):
        
        if not self.direction:
            self.position[0] += 1
        else:
            self.position[1] += 1
        
    def turn(self):
        
        self.direction = int(not self.direction)
        self.update_position()
    
    
class Road:
    
    def __init__(self, direction, position, dimension):
        
        self.alpha = alpha
        self.beta = beta
        self.dimension = dimension
        
        self.grid = np.zeros((self.dimension,))
        self.particles = []
        self.direction = direction
        self.position = position
    
    def move_particle(self, particle, turn=False):
        
        rv = np.random.random()
        if rv > particle.p:
            return None
        if turn:
            particle.turn()
        if not self.direction:
            curr_pos = particle.position[0]
        else:
            curr_pos = particle.position[1]
        if not curr_pos == self.dimension - 1 and not self.grid[curr_pos + 1]:
            particle.update_position()
            self.update()
            return '{} moved to {} at time '.format(particle.id, particle.position)
        return None
    
    def update(self):
        
        self.grid = np.zeros((self.dimension,))
        if not self.direction:
            for p in self.particles:
                self.grid[p.position[0]] = 1
                
        else:
            for p in self.particles:
                self.grid[p.position[1]] = 1
                
    def add_particle(self):
        
        if not self.grid[0]:
            new_particle = Particle2D(self, self.direction)
            self.particles.append(new_particle)
            self.update()
            return new_particle, '{} added at {} at time '.format(new_particle.id, new_particle.position)
        return None, None
        
    def remove_particle(self):
        
        if self.grid[-1]:
            curr_particle = self.particles.pop(-1)
            self.update()
            return curr_particle, '{} exited from {} at time '.format(curr_particle.id, curr_particle.position)
        return None, None
        
    
class Intersection:
    
    def __init__(self, position):
        
        self.theta = theta
        
        self.position = position
        self.light = 0
        
    def change_light(self):
        
        self.light = int(not self.light)
    
    
class Environment2D:
    
    def __init__(self, num_horz_roads, num_vert_roads, dimension):
        
        self.dimension = dimension
        
        self.entrances = 0
        self.exits = 0
        
        horizontal_positions = np.random.choice(range(1,dimension-1), num_horz_roads, replace=False)
        vertical_positions = np.random.choice(range(1,dimension-1), num_vert_roads, replace=False)
        
        self.horz_roads = [Road(0, p, dimension) for p in horizontal_positions]
        self.vert_roads = [Road(1, p, dimension) for p in vertical_positions]
        
        grid = np.empty((dimension, dimension), dtype='object')
        grid[:] = ' '
        for road in self.horz_roads:
            grid[road.position, :] = road.grid
        for road in self.vert_roads:
            grid[:, road.position] = road.grid
            
        self.grid = grid
        
        self.intersections = []
        for y in self.horz_roads:
            for x in self.vert_roads:
                new_intersection = Intersection((x.position, y.position))
                self.intersections.append(new_intersection)
                
    def get_param(self):
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        return move_param + add_param + remove_param + light_param
    
    def choose_action(self):
        
        actions = ['move', 'add', 'remove', 'light']
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        probs = [move_param, add_param, remove_param, light_param]
        probs = probs / np.sum(probs)
        
        return np.random.choice(actions, p=probs)
        
    def move_particle(self):
        
        # choose road
        horz_p = [sum([p.gamma for p in road.particles]) for road in self.horz_roads]
        vert_p = [sum([p.gamma for p in road.particles]) for road in self.vert_roads]
        probs = np.array(horz_p + vert_p)
        probs = probs / np.sum(probs)
        road = np.random.choice(self.horz_roads + self.vert_roads, p=probs)
        
        # choose particle
        probs = np.array([p.gamma for p in road.particles])
        probs = probs / np.sum(probs)
        particle = np.random.choice(road.particles, p=probs)
        
        # update
        
        turn = False
        if particle.position in [i.position for i in self.intersections]:
            intersection = [i for i in self.intersections if i.position == particle.position][0]
            if intersection.light != particle.direction:
                return None
            else:
                rv = np.random.random()
                if rv < particle.z:
                    turn = True
        
        result = road.move_particle(particle, turn)
#         if result == None:
#             self.sg_stopped += 1
#         self.move_opps += 1
        self.update_grid(road)
        return particle, result
        
    def add_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = np.array([road.alpha for road in roads])
        probs = probs / np.sum(probs)
        road = np.random.choice(roads, p=probs)
        particle, result = road.add_particle()
        if result:
            self.update_grid(road)
            self.entrances += 1
        return particle, result
        
    def remove_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = np.array([road.beta for road in roads])
        probs = probs / np.sum(probs)
        road = np.random.choice(roads, p=probs)
        particle, result = road.remove_particle()
        if result:
            self.update_grid(road)
            self.exits += 1
        return particle, result
        
    def update_light(self):
        
        probs = np.array([i.theta for i in self.intersections])
        probs = probs / np.sum(probs)
        intersection = np.random.choice(self.intersections, p=probs)
        intersection.change_light()
        
        return 'light switched at {} at '.format(intersection.position)
        
    def update_grid(self, road):
        
        if not road.direction:
            self.grid[road.position, :] = road.grid
        else:
            self.grid[:, road.position] = road.grid

    
class Simulation:
    
    def __init__(self):
        
        self.env = Environment2D(2, 2, 20)
        self.clock = 0
        self.log = []
        self.travel_times = {}
        self.time_of_cars = []
        self.cell_speeds = []
        self.num_cars = [0]
        self.stop_and_go_time = 0
        self.move_opps = 0
        self.sg_stopped = 0
        self.sg_time_lapse = 0 
        self.sg_interval_len = 5 # constant
        self.sg_max_stoppage_percent = .3 #constant
        
    def visualize(self):
        
        clear_output(wait=True)
        print(self.env.grid)
    
    def take_action(self):
        
        param = self.env.get_param()
        time_lapse = np.random.exponential(scale=1/param)
        time.sleep(time_lapse)
        
        #stop and go 2d
        if self.sg_time_lapse != 0 and self.sg_time_lapse >= self.sg_interval_len:
            if self.move_opps > 0 and self.sg_stopped / self.move_opps >= self.sg_max_stoppage_percent:
                self.stop_and_go_time += self.sg_time_lapse
            self.sg_time_lapse = 0
            self.sg_stopped = 0
            self.move_opps = 0
        else:
            self.sg_time_lapse += time_lapse
        
        self.clock += time_lapse
        
        action = self.env.choose_action()
        
        if action == 'move':
            particle, result = self.env.move_particle()
            if particle:
                self.cell_speeds.append(1 / (self.clock - particle.time))
                particle.time = self.clock
            else:
                self.sg_stopped += 1
            self.move_opps += 1
                
        elif action == 'add':
            particle, result = self.env.add_particle()
            if particle:
                cars = sum([len(road.particles) for road in self.env.horz_roads]) + sum([len(road.particles) for road in self.env.vert_roads])
                self.num_cars.append(cars)
                particle.time = self.clock
                self.travel_times[particle] = [self.clock, None]
                self.time_of_cars.append(self.clock)
        elif action == 'remove':
            particle, result = self.env.remove_particle()
            if particle:
                cars = sum([len(road.particles) for road in self.env.horz_roads]) + sum([len(road.particles) for road in self.env.vert_roads])
                self.num_cars.append(cars)
                particle.time = self.clock
                self.travel_times[particle][1] = self.clock
                self.time_of_cars.append(self.clock)
        elif action == 'light':
            result = self.env.update_light()
            
        return result
        
    def run(self):
        
        while self.clock < 10:
            result = self.take_action()
            if result:
                self.log.append(result + str(self.clock))
            self.visualize()
            
        entrances = self.env.entrances
        exits = self.env.exits
        travel_times = self.travel_times.values()
        cell_speeds = self.cell_speeds
        time_of_cars = self.time_of_cars
        num_cars = self.num_cars
        
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
            
            
        time_of_cars.append(self.clock)
        t = len(time_of_cars)
        for i in range(1, t):
            time_of_cars[t - i] = time_of_cars[t - i] - time_of_cars[t - i - 1]
        average_num_cars = np.average(num_cars, weights=np.array(time_of_cars) / self.clock)
        print('Average number of cars: {}'.format(round(average_num_cars, 2)))
        print('Stop and Go Time Percentage: {}'.format(self.stop_and_go_time/self.clock))
        
        [print(line) for line in self.log]

### Multiple Lanes Environment:

In [226]:
class ParticleManyLane(Particle):
    def __init__(self, time, lane, gamma=gamma, p=p):
        Particle.__init__(self, time, gamma, p)
        self._position = [lane, 0] #first index lane, second index position
        
    def update_position(self, time, value=(0, 1)):
        self._time = time
        self._position[0] += value[0]
        self._position[1] += value[1]
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_position(self):
        return self._position[0], self._position[1]
    
class EnvManyLane(Environment):
    def __init__(self, n=n, alpha=alpha, beta=beta, lanes=1, delta=0.5):
        Environment.__init__(self, n, alpha, beta)
        self.lanes = lanes
        self.occupied_positions = np.zeros((lanes, n))
        self.delta = delta # tendency to change lanes
        #first index lane, second index position
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 
        curr_time = 0
        entrances = 0
        exits = 0
        actions = []
        travel_times = []
        cell_speeds = []
        num_cars = [[0] for _ in range(self.lanes)]
        time_of_cars = [[] for _ in range(self.lanes)]
        
        stop_and_go_time = 0
        sg_time_lapse = 0 
        sg_interval_len = 5 # constant
        sg_max_stoppage_percent = .3 #constant
        move_opps = [0]
        sg_stopped = [0]
        
        while curr_time < time_len:
            clear_output(wait=True)
            #print(self.occupied_positions)
            print(np.array2string(np.asarray(self.occupied_positions, dtype='object'), formatter={'object': format_func}))  #new format multi
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.lanes * self.alpha + self.lanes * self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] +  [self.lanes * self.alpha, self.lanes * self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            
            #stop and go multi-lane
            if sg_time_lapse != 0 and sg_time_lapse >= sg_interval_len:
                if move_opps[0] > 0 and sg_stopped[0] / move_opps[0] >= sg_max_stoppage_percent:
                    stop_and_go_time += sg_time_lapse
                sg_time_lapse = 0
                sg_stopped = [0]
                move_opps = [0]
            else:
                sg_time_lapse += time_lapse


            curr_time += time_lapse
            if particle == 'entrance':
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, 0]:
                    continue
                else:
                    new_particle = ParticleManyLane(curr_time, lane, gamma=np.random.normal(gamma, gamma_sd), p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[lane, 0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    travel_times.append([curr_time, None])
                    num_cars[lane].append(np.sum(self.occupied_positions[lane, :]))
                    time_of_cars[lane].append(curr_time)
                    
            elif particle == 'exit':
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, self.n-1]: 
                    for i in range(len(self.particles)):
                        particle = self.particles[i]
                        if particle.get_position() == (lane, self.n-1):
                            end_particle = particle
                            self.particles.pop(i)
                            break
                    self.occupied_positions[lane, self.n-1] = 0
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    travel_times[end_particle.id][1] = curr_time
                    num_cars[lane].append(np.sum(self.occupied_positions[lane, :]))
                    time_of_cars[lane].append(curr_time)
                    
                else:
                    continue
            else:
                particle_lane, particle_pos = particle.get_position()
                options = []
                if particle_pos != self.n - 1 and self.occupied_positions[particle_lane, particle_pos + 1] != 1:
                    options.append((0, 1)) # advance
                if particle_lane - 1 >= 0 and self.occupied_positions[particle_lane - 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane - 1, particle_pos] == 0:
                    options.append((-1, 0)) # left
                if particle_lane + 1 < self.lanes and self.occupied_positions[particle_lane + 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane + 1, particle_pos] == 0:
                    options.append((1, 0)) # right
                    
                if len(options) == 0:
                    continue
                else:
                    probabilities = []
                    # see which options are available and pick with probability proportional to the number of open spaces
                    # proportional to number of cars in front
                    if (0, 1) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane, particle_pos:])) * (1 - self.delta))
                    if (-1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane - 1, particle_pos:])) * self.delta)
                    if (1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane + 1, particle_pos:])) * self.delta)
                    probabilities = probabilities / sum(probabilities)
                    # choose option and update model
                    option = options[np.random.choice(len(options), p=probabilities)]
                    self.occupied_positions[particle_lane, particle_pos] = 0
                    self.occupied_positions[particle_lane + option[0], particle_pos + option[1]] = 1
                    if option == (0, 1):
                        cell_speeds.append(1 / (curr_time - particle._time))
#                         sg_speeds.append(1 / (curr_time - particle._time))
                        particle.update_position(curr_time, (0, 1))
                    else:
                        num_cars[particle_lane].append(num_cars[particle_lane][-1] - 1)
                        time_of_cars[particle_lane].append(curr_time)
                        particle.update_position(particle._time, (option))
                        particle_lane, particle_pos = particle.get_position()
                        num_cars[particle_lane].append(num_cars[particle_lane][-1] + 1)
                        time_of_cars[particle_lane].append(curr_time)
                        
                    actions += [(curr_time, particle.id, particle.get_position())]
                          
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
#             print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {} seconds'.format(round(sum(travel_times) / len(travel_times), 3)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {} cells per second'.format(round(sum(cell_speeds) / len(cell_speeds), 3)))
        for lane in time_of_cars:
            lane.append(curr_time)
        
        for lane in time_of_cars:
            t = len(lane)
            for i in range(1, t):
                lane[t - i] = lane[t - i] - lane[t - i - 1]
        average_num_cars = [np.average(n, weights=np.array(ti) / curr_time) for n, ti in zip(num_cars, time_of_cars)]
        average_num_cars = list(map(lambda x: round(x, 2), average_num_cars))
        print('Average number of cars in each lane: {}'.format(average_num_cars))
        print('Stop and Go Time Percentage: {}'.format(stop_and_go_time/time_len))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(round(entry[0], 2)), end = " ")
            print('Particle: {}'.format(entry[1]), end = " ")
            print('New lane, position: ({},{})'.format(entry[2][0], entry[2][1]), end = " ")
            print('\n')

### Run simulator:

In [ ]:
env1 = Environment(n=n, alpha=alpha, beta=beta)
env1.run_simulation(time_len = 60)

In [222]:
sim = Simulation()
sim.run()

In [228]:
env2 = EnvManyLane(n=n, alpha=alpha, beta=beta, lanes=2)
env2.run_simulation(time_len = 2)

[[R _ _ _ _ _ _ _ _ _]
 [R _ _ _ _ _ _ _ _ _]]

 done
Summary:
Total entrances: 2
Total exits: 0
No cars reached the end
Average number of cars in each lane: [0.73, 0.89]
Stop and Go Time Percentage: 0.0

 Log:
Time: 0.22 Particle: 0 New lane, position: (1,0) 

Time: 0.54 Particle: 1 New lane, position: (0,0) 

